# 프로젝트 - Movielens 영화 추천 실습

이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

cloud jupyter 를 열고 aiffel/recommendata_iu/ 디렉토리에 python3로 ipynb파일을 생성 해 주세요 jupyter notebook 환경에서 프로젝트 코드를 작성하고 제출하시면 됩니다.

이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
별점을 시청횟수로 해석해서 생각하겠습니다.
또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.
Cloud Storage에 미리 업로드 된 ml-1m폴더 내 파일을 심볼릭 링크로 개인 storage에 연결 해 줍니다.

Cloud shell에서 아래 명령어를 입력해주세요.

In [1]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/EXP_09_recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [4]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/EXP_09_recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# movie_id는 3952까지 존재
movies.tail()

,movie_id,title,genre
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller


여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.



### 2) 분석해 봅시다.

 - ratings에 있는 유니크한 영화 개수
 - ratings에 있는 유니크한 사용자 수
 - 가장 인기 있는 영화 30개(인기순)


In [8]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

# ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()


# ratings 와 movies 데이터 프레임 합치기
# movie_id를 key로 ratings와 movies 데이터프레임을 조인
ratings = ratings.merge(movies, how='left', on='movie_id')
ratings.groupby('title')['user_id'].count().sort_values(ascending=False)[:30]

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [10]:
# ratings의 데이터타입 체크
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836478 entries, 0 to 836477
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user_id    836478 non-null  int64 
 1   movie_id   836478 non-null  int64 
 2   counts     836478 non-null  int64 
 3   timestamp  836478 non-null  int64 
 4   title      836478 non-null  object
 5   genre      836478 non-null  object
dtypes: int64(4), object(2)
memory usage: 44.7+ MB


#### 결측치

In [12]:
# 결측값 존재 여부를 체크해보자
ratings.isnull().sum()

user_id      0
movie_id     0
counts       0
timestamp    0
title        0
genre        0
dtype: int64

#### 3.2 Unique Genre

In [13]:
# genre는 pipe-separated(|)로 구분 되어있으므로 unique한 genre가 무엇이 있는지 확인해보자
genres = list(movies['genre'].value_counts().keys())
genres[:20]

['Drama',
 'Comedy',
 'Horror',
 'Comedy|Drama',
 'Comedy|Romance',
 'Drama|Romance',
 'Documentary',
 'Thriller',
 'Action',
 'Drama|Thriller',
 'Action|Thriller',
 "Children's|Comedy",
 'Crime|Drama',
 'Drama|War',
 'Romance',
 'Action|Drama',
 "Animation|Children's",
 'Comedy|Drama|Romance',
 'Western',
 'Horror|Sci-Fi']

In [23]:
# genre는 pipe-separated(|)로 구분 되어있으므로 unique한 genre가 무엇이 있는지 확인해보자
genres = list(movies['genre'].value_counts().keys())
genres[:20]

['Drama',
 'Comedy',
 'Horror',
 'Comedy|Drama',
 'Comedy|Romance',
 'Drama|Romance',
 'Documentary',
 'Thriller',
 'Action',
 'Drama|Thriller',
 'Action|Thriller',
 "Children's|Comedy",
 'Crime|Drama',
 'Drama|War',
 'Romance',
 'Action|Drama',
 "Animation|Children's",
 'Comedy|Drama|Romance',
 'Western',
 'Horror|Sci-Fi']

In [24]:
from collections import Counter

result = [] # 중복을 포함해서 genre 출현 빈도를 count
for genre in ratings['genre']:
    result.extend(genre.split('|'))

In [25]:
genre_counter = Counter(result)
genre_counter

Counter({'Drama': 313261,
         'Animation': 37609,
         "Children's": 57674,
         'Musical': 35661,
         'Romance': 125927,
         'Comedy': 293891,
         'Action': 209494,
         'Adventure': 108818,
         'Fantasy': 29205,
         'Sci-Fi': 126089,
         'War': 61226,
         'Thriller': 158343,
         'Crime': 68889,
         'Mystery': 34417,
         'Western': 17668,
         'Horror': 55488,
         'Film-Noir': 17166,
         'Documentary': 7162})

In [26]:
sorted_genre_counter = genre_counter.most_common()
sorted_genre_counter

[('Drama', 313261),
 ('Comedy', 293891),
 ('Action', 209494),
 ('Thriller', 158343),
 ('Sci-Fi', 126089),
 ('Romance', 125927),
 ('Adventure', 108818),
 ('Crime', 68889),
 ('War', 61226),
 ("Children's", 57674),
 ('Horror', 55488),
 ('Animation', 37609),
 ('Musical', 35661),
 ('Mystery', 34417),
 ('Fantasy', 29205),
 ('Western', 17668),
 ('Film-Noir', 17166),
 ('Documentary', 7162)]

#### 데이터 시각화 by using plotly

In [27]:
!pip install plotly

# Horizontal bar

import pandas as pd
import plotly.express as px

import plotly.io as pio
pio.renderers.default = 'notebook_connected'

sorted_genre_counter_df = pd.DataFrame(sorted_genre_counter, columns=['genre', 'count'])

fig = px.bar(sorted_genre_counter_df, x='count', y='genre', orientation='h')

fig.show()

     |████████████████████████████████| 23.9 MB 4.6 MB/s eta 0:00:01
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [31]:
# Pie chart
fig = px.pie(sorted_genre_counter_df, values='count', names='genre', title='Pie chart for Genre')
fig.show()

In [32]:
# helper function
def get_unique_genre(genres):
    result = [] # unique 장르를 담을 list
    for genre in genres:
        result.extend(genre.split('|'))
    return sorted(set(result))

In [33]:
# unique한 genre는 다음과 같다.
unique_genres = get_unique_genre(genres)
unique_genres.sort
print(unique_genres)

['Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']


#### Timestamp

In [35]:
ratings.head()

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,2000-12-31,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,2000-12-31,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,2000-12-31,My Fair Lady (1964),Musical|Romance
3,1,3408,4,2000-12-31,Erin Brockovich (2000),Drama
4,1,2355,5,2001-01-06,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [34]:
from datetime import datetime
import time
s = int('978300760')

datetime.fromtimestamp(s).strftime('%Y-%m-%d')

# timestamp를 %Y-%m-%d 형식으로 변경해봅니다.
# ratings.rename(columns={'timestamp' : 'date'}, inplace=True)
ratings['timestamp'] = ratings['timestamp'].apply(lambda s : datetime.fromtimestamp(int(s)).strftime('%Y-%m-%d'))


ratings.head()

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,2000-12-31,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,2000-12-31,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,2000-12-31,My Fair Lady (1964),Musical|Romance
3,1,3408,4,2000-12-31,Erin Brockovich (2000),Drama
4,1,2355,5,2001-01-06,"Bug's Life, A (1998)",Animation|Children's|Comedy


#### year

In [37]:
ratings['year'] = ratings['title'].str.extract('.*\((.*)\).*',expand = False).astype(int)

ratings.head()

,user_id,movie_id,counts,timestamp,title,genre,year
0,1,1193,5,2000-12-31,One Flew Over the Cuckoo's Nest (1975),Drama,1975
1,1,661,3,2000-12-31,James and the Giant Peach (1996),Animation|Children's|Musical,1996
2,1,914,3,2000-12-31,My Fair Lady (1964),Musical|Romance,1964
3,1,3408,4,2000-12-31,Erin Brockovich (2000),Drama,2000
4,1,2355,5,2001-01-06,"Bug's Life, A (1998)",Animation|Children's|Comedy,1998


In [42]:
# 가장 인기있는 영화 top 10을 시각화 해봅니다.

import plotly.graph_objects as go

movies_top10 = ratings.groupby('title')['user_id'].count().sort_values(ascending=False)[:10]

fig = go.Figure(data=go.Scatter(
    x=movies_top10.index,
    y=movies_top10.values,
    mode='markers',
    marker=dict(size=[100,90,80,70,60,50,40,30,20,10],
                color=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
))

fig.show()

In [44]:
# 90년대 인기 영화 top 10을 시각화

cond1 = 'year >= 1990 and year <= 2000' 
movies_90s = ratings.query(cond1)
movies_90s_top10 = movies_90s.groupby('title')['user_id'].count().sort_values(ascending=False)[:10]

fig1 = go.Figure(data=go.Scatter(
    x=movies_90s_top10.index,
    y=movies_90s_top10.values,
    mode='markers',
    marker=dict(size=[100,90,80,70,60,50,40,30,20,10],
                color=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
))

fig1.show()

In [45]:
# 80년대 인기 영화 top 10

cond2 = 'year >= 1980 and year < 1990' 
movies_80s = ratings.query(cond2)
movies_80s_top10 = movies_80s.groupby('title')['user_id'].count().sort_values(ascending=False)[:10]

fig2 = go.Figure(data=go.Scatter(
    x=movies_80s_top10.index,
    y=movies_80s_top10.values,
    mode='markers',
    marker=dict(size=[100,90,80,70,60,50,40,30,20,10],
                color=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
))

fig2.show()

In [46]:

# 70년대 인기 영화 top 10

cond3 = 'year >= 1970 and year < 1980' 
movies_70s = ratings.query(cond3)
movies_70s_top10 = movies_70s.groupby('title')['user_id'].count().sort_values(ascending=False)[:10]

fig3 = go.Figure(data=go.Scatter(
    x=movies_70s_top10.index,
    y=movies_70s_top10.values,
    mode='markers',
    marker=dict(size=[100,90,80,70,60,50,40,30,20,10],
                color=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
))

fig3.show()

#### rating

In [53]:
avg_ratings = ratings[['movie_id', 'counts', 'year']].groupby('movie_id').mean()

fig = px.scatter(avg_ratings, x="year", y="counts", color='counts')
fig.show()

### 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

user_id : 기존 id가 6040까지 있으므로 내 id는 6041으로 설정한다.
movie_id : 내가 선택할 영화의 id가 리스트에 존재한다면 해당 id로 설정하고 그렇지 않다면 3952 이후로 설정
count : 내가 좋아하는 영화이므로 5점을 부여
title : title 형식에 맞게 '제목 (연도)'로 전처리
genre : 장르가 여러개 존재할 경우 |로 연결

In [54]:
# 2000년 이전 영화 중에서 내가 좋아하는 영화 5가지를 선택한다.

# 라이언 일병 구하기 Saving Private Ryan, 1998 War, Action, Drama
# 인생은 아름다워 Life Is Beautiful, 1997, Drama, Comedy
# 쇼생크 탈출 The Shawshank Redemption, 1994, Drama
# 타이타닉 Titanic, 1997, Romance, Drama
# 매트릭스 The Matrix, 1999 Sci-Fi, Action

# 영화 하나 하나 검색하며 title이 일치하는지 여부를 확인한다.

favorite_movies = ['Saving Private Ryan (1998)',
                   'Life Is Beautiful (1997)',
                   'Star Wars: Episode IV - A New Hope (1977)',
                   'Titanic (1997)',
                   'The Matrix (1999)']

In [55]:
# 라이언 일병 구하기
movie_name = 'Saving Private Ryan (1998)'
ratings[ratings['title'].apply(lambda x : x == movie_name)]

,user_id,movie_id,counts,timestamp,title,genre,year
48,1,2028,5,2000-12-31,Saving Private Ryan (1998),Action|Drama|War,1998
102,2,2028,4,2000-12-31,Saving Private Ryan (1998),Action|Drama|War,1998
221,4,2028,5,2000-12-31,Saving Private Ryan (1998),Action|Drama|War,1998
473,7,2028,5,2000-12-31,Saving Private Ryan (1998),Action|Drama|War,1998
571,8,2028,5,2000-12-31,Saving Private Ryan (1998),Action|Drama|War,1998
...,...,...,...,...,...,...,...
834644,6027,2028,5,2000-04-26,Saving Private Ryan (1998),Action|Drama|War,1998
835008,6033,2028,5,2000-04-26,Saving Private Ryan (1998),Action|Drama|War,1998
835862,6036,2028,5,2000-04-26,Saving Private Ryan (1998),Action|Drama|War,1998
836059,6037,2028,4,2000-04-26,Saving Private Ryan (1998),Action|Drama|War,1998


In [56]:
# 인생은 아름다워
# Life Is Beautiful (1997)으로는 인덱싱이 되지 않는다'
# 제목을 'Life Is Beautiful (La Vita è bella) (1997)'로 수정한다.

movie_name = 'Life Is Beautiful (1997)'
favorite_movies[1] = 'Life Is Beautiful (La Vita è bella) (1997)'
ratings[ratings['title'].apply(lambda x : x[:4] == 'Life')]

,user_id,movie_id,counts,timestamp,title,genre,year
595,8,2324,3,2000-12-31,Life Is Beautiful (La Vita è bella) (1997),Comedy|Drama,1997
698,9,2324,5,2000-12-31,Life Is Beautiful (La Vita è bella) (1997),Comedy|Drama,1997
885,10,2324,5,2000-12-31,Life Is Beautiful (La Vita è bella) (1997),Comedy|Drama,1997
1673,17,2377,5,2000-12-30,Lifeforce (1985),Horror|Sci-Fi,1985
2078,19,2324,5,2001-01-03,Life Is Beautiful (La Vita è bella) (1997),Comedy|Drama,1997
...,...,...,...,...,...,...,...
835133,6035,2324,5,2000-04-26,Life Is Beautiful (La Vita è bella) (1997),Comedy|Drama,1997
835728,6036,2324,4,2000-04-26,Life Is Beautiful (La Vita è bella) (1997),Comedy|Drama,1997
835756,6036,2377,3,2000-04-26,Lifeforce (1985),Horror|Sci-Fi,1985
836038,6037,2324,4,2000-04-26,Life Is Beautiful (La Vita è bella) (1997),Comedy|Drama,1997


In [62]:
# 스타워즈

movie_name = 'Star Wars: Episode IV - A New Hope (1977)'
movies[2] = 'Star Wars: Episode IV - A New Hope (1977)'
movies[movies['title'].apply(lambda x : (x[:7] == 'Star Wa'))]

,movie_id,title,genre,2,1
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode IV - A New Hope (1977)
1178,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War,Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode IV - A New Hope (1977)
1192,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War,Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode IV - A New Hope (1977)
2559,2628,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Fantasy|Sci-Fi,Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode IV - A New Hope (1977)


In [63]:
# 타이타닉 
movie_name = 'Titanic (1997)'
ratings[ratings['title'].apply(lambda x : x == movie_name)]

,user_id,movie_id,counts,timestamp,title,genre,year
27,1,1721,4,2000-12-31,Titanic (1997),Drama|Romance,1997
557,8,1721,5,2000-12-31,Titanic (1997),Drama|Romance,1997
672,9,1721,5,2000-12-31,Titanic (1997),Drama|Romance,1997
1027,10,1721,3,2001-01-10,Titanic (1997),Drama|Romance,1997
1934,18,1721,4,2000-12-30,Titanic (1997),Drama|Romance,1997
...,...,...,...,...,...,...,...
833544,6016,1721,3,2000-04-28,Titanic (1997),Drama|Romance,1997
834214,6023,1721,5,2000-04-26,Titanic (1997),Drama|Romance,1997
834500,6025,1721,4,2000-04-26,Titanic (1997),Drama|Romance,1997
834642,6027,1721,4,2000-04-26,Titanic (1997),Drama|Romance,1997


In [64]:
# 매트릭스
# 매트릭스는 Matrix, The (1999)로 수정한다.
movie_name = 'The Matrix (1999)'
favorite_movies[4] = 'Matrix, The (1999)' 
ratings[ratings['title'].apply(lambda x : x[:6] == 'Matrix')]

,user_id,movie_id,counts,timestamp,title,genre,year
127,2,2571,4,2000-12-31,"Matrix, The (1999)",Action|Sci-Fi|Thriller,1999
286,5,2571,5,2000-12-31,"Matrix, The (1999)",Action|Sci-Fi|Thriller,1999
459,7,2571,5,2000-12-31,"Matrix, The (1999)",Action|Sci-Fi|Thriller,1999
517,8,2571,5,2000-12-31,"Matrix, The (1999)",Action|Sci-Fi|Thriller,1999
640,9,2571,5,2000-12-31,"Matrix, The (1999)",Action|Sci-Fi|Thriller,1999
...,...,...,...,...,...,...,...
834757,6030,2571,5,2000-04-26,"Matrix, The (1999)",Action|Sci-Fi|Thriller,1999
834829,6031,2571,5,2000-04-26,"Matrix, The (1999)",Action|Sci-Fi|Thriller,1999
835072,6035,2571,5,2000-04-26,"Matrix, The (1999)",Action|Sci-Fi|Thriller,1999
835456,6036,2571,3,2000-04-26,"Matrix, The (1999)",Action|Sci-Fi|Thriller,1999


In [65]:
favorite_movies

['Saving Private Ryan (1998)',
 'Life Is Beautiful (La Vita è bella) (1997)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Titanic (1997)',
 'Matrix, The (1999)']

In [66]:
my_movies = {
    'user_id' : [6041] * 5,
    'movie_id' : [2028, 2324, 318, 1721, 2571],
    'count' : [5]*5,
    'title' : favorite_movies,
    'genre': ['Action|Drama|War',
             'Comedy|Drama',
             'Drama',
             'Drama|Romance',
             'Action|Sci-Fi|Thriller']
}

In [67]:
my_movies_df = pd.DataFrame(my_movies)

In [68]:
ratings = ratings.append(my_movies_df, ignore_index=True)

In [69]:
ratings.tail(10)

,user_id,movie_id,counts,timestamp,title,genre,year,count
836473,6040,1090,3.0,2000-04-26,Platoon (1986),Drama|War,1986.0,NaN
836474,6040,1094,5.0,2000-04-25,"Crying Game, The (1992)",Drama|Romance|War,1992.0,NaN
836475,6040,562,5.0,2000-04-25,Welcome to the Dollhouse (1995),Comedy|Drama,1995.0,NaN
836476,6040,1096,4.0,2000-04-26,Sophie's Choice (1982),Drama,1982.0,NaN
836477,6040,1097,4.0,2000-04-26,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi,1982.0,NaN
836478,6041,2028,NaN,NaN,Saving Private Ryan (1998),Action|Drama|War,NaN,5.0
836479,6041,2324,NaN,NaN,Life Is Beautiful (La Vita è bella) (1997),Comedy|Drama,NaN,5.0
836480,6041,318,NaN,NaN,Star Wars: Episode IV - A New Hope (1977),Drama,NaN,5.0
836481,6041,1721,NaN,NaN,Titanic (1997),Drama|Romance,NaN,5.0
836482,6041,2571,NaN,NaN,"Matrix, The (1999)",Action|Sci-Fi|Thriller,NaN,5.0


In [70]:
# 이제 필요한 칼럼만 남기고 제거한다.
ratings.drop(['timestamp', 'genre', 'year'], axis=1, inplace=True)

In [91]:
ratings

,user_id,movie_id,counts,title,count
0,0,1193,5.0,0,NaN
1,0,661,3.0,1,NaN
2,0,914,3.0,2,NaN
3,0,3408,4.0,3,NaN
4,0,2355,5.0,4,NaN
...,...,...,...,...,...
836478,6039,2028,NaN,48,5.0
836479,6039,2324,NaN,450,5.0
836480,6039,318,NaN,44,5.0
836481,6039,1721,NaN,27,5.0


In [72]:

# user_id와 title의 unique 값을 저장
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

# unique 값의 개수만큼 user_id와 title을 indexing
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [73]:
# 데이터 컬럼 내 값을 indexing된 값으로 교체

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

ratings

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,counts,title,count
0,0,1193,5.0,0,NaN
1,0,661,3.0,1,NaN
2,0,914,3.0,2,NaN
3,0,3408,4.0,3,NaN
4,0,2355,5.0,4,NaN
...,...,...,...,...,...
836478,6039,2028,NaN,48,5.0
836479,6039,2324,NaN,450,5.0
836480,6039,318,NaN,44,5.0
836481,6039,1721,NaN,27,5.0


In [84]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['title'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings['user_id'], ratings['title'])), shape=(num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.float64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [85]:
!pip install implicit

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [86]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [87]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=30, dtype=np.float32)

In [88]:
# als 모델은 input으로 item X user 꼴의 matrix를 받기 때문에 Transpose 해준다.
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.float64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [89]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/30 [00:00<?, ?it/s]

ModelFitError: NaN encountered in factors

In [ ]:
my_id = user_to_idx[6041] # 내가 새로 추가한 유저 아이디
saving_ryan = title_to_idx['Saving Private Ryan (1998)']

my_vector, saving_ryan_vector = als_model.user_factors[my_id], als_model.item_factors[saving_ryan]

In [80]:
my_vector

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32)

In [81]:
matrix_vector

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32)

In [82]:
# 라이언 일병 구하기를 좋아한다!
favorite_movie = 'Saving Private Ryan (1998)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(1203, nan),
 (1192, nan),
 (1217, nan),
 (1206, nan),
 (1207, nan),
 (1208, nan),
 (1209, nan),
 (1210, nan),
 (1211, nan),
 (1212, nan),
 (1213, nan),
 (1214, nan),
 (1215, nan),
 (1216, nan),
 (3627, nan)]

In [92]:

# 라이언 일병 구하기와 유사한 추천 영화 15개
# 쇼생크 탈출이 두개 보이고.. 쉰들러 리스트는 좋아하는 영화다!
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Something to Talk About (1995)',
 'How Stella Got Her Groove Back (1998)',
 'Blue Chips (1994)',
 'Hope Floats (1998)',
 'Milk Money (1994)',
 'Miami Rhapsody (1995)',
 'Oxygen (1999)',
 'Blame It on Rio (1984)',
 'Fools Rush In (1997)',
 'Tin Cup (1996)',
 'Nine 1/2 Weeks (1986)',
 'Random Hearts (1999)',
 'Metropolitan (1990)',
 'Client, The (1994)',
 'Identification of a Woman (Identificazione di una donna) (1982)']

In [93]:
# 비슷한 영화를 알려주는 함수
def get_similar_movie(movie_title):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [94]:
# 쉰들러리스트와 유사한 영화
get_similar_movie("Schindler's List (1993)")

['Metropolitan (1990)',
 'Fools Rush In (1997)',
 'Nine 1/2 Weeks (1986)',
 'Blue Chips (1994)',
 'Hope Floats (1998)',
 'Blame It on Rio (1984)',
 'Oxygen (1999)',
 'Miami Rhapsody (1995)',
 'Milk Money (1994)',
 'Identification of a Woman (Identificazione di una donna) (1982)']

In [95]:
# 레옹도 좋아한다. 이중에서 SEVEN도 좋아하는 영화이다.
get_similar_movie('Professional, The (a.k.a. Leon: The Professional) (1994)')

['Metropolitan (1990)',
 'Fools Rush In (1997)',
 'Nine 1/2 Weeks (1986)',
 'Blue Chips (1994)',
 'Hope Floats (1998)',
 'Blame It on Rio (1984)',
 'Oxygen (1999)',
 'Miami Rhapsody (1995)',
 'Milk Money (1994)',
 'Identification of a Woman (Identificazione di una donna) (1982)']

In [97]:

# 나를 위한 영화 추천
for_me = user_to_idx[6041]
recommended_movie = als_model.recommend(for_me, csr_data, N=20, filter_already_liked_items=True)
recommended_movie

[(1198, nan),
 (1201, nan),
 (1197, nan),
 (1199, nan),
 (1204, nan),
 (1205, nan),
 (1196, nan),
 (1195, nan),
 (1194, nan),
 (1193, nan),
 (1203, nan),
 (1192, nan),
 (1217, nan),
 (1206, nan),
 (1207, nan),
 (1208, nan),
 (1209, nan),
 (1210, nan),
 (1211, nan),
 (1212, nan)]

In [98]:
# 인덱스를 영화 제목으로 변환
[idx_to_movie[i[0]] for i in recommended_movie]

['Diner (1982)',
 'Meatballs (1979)',
 'Honeymoon in Vegas (1992)',
 'Substitute, The (1996)',
 'Lawn Dogs (1997)',
 'Bad Girls (1994)',
 'Net, The (1995)',
 'I.Q. (1994)',
 'Fatal Attraction (1987)',
 'Heathers (1989)',
 'Something to Talk About (1995)',
 'How Stella Got Her Groove Back (1998)',
 'Blue Chips (1994)',
 'Hope Floats (1998)',
 'Milk Money (1994)',
 'Miami Rhapsody (1995)',
 'Oxygen (1999)',
 'Blame It on Rio (1984)',
 'Fools Rush In (1997)',
 'Tin Cup (1996)']

In [99]:
# 사용자가 기록을 남긴 데이터가 특정 영화 추천에 기여한 정도를 확인
recommended = movie_to_idx['Star Wars: Episode IV - A New Hope (1977)']
explain = als_model.explain(for_me, csr_data, itemid=recommended)

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

ValueError: array must not contain infs or NaNs

### 4) CSR matrix를 직접 만들어 봅시다

In [ ]:

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape= (num_user, num_artist))
csr_data

### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.


### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.


### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.